In [ ]:
import ipywidgets as widgets
from functools import partial
import shutil
import yaml
import os

LABELS_PATH="/workspaces/motion/labels.yaml"
INCOMING_IMAGES_DIR = "/workspaces/motion/images/esp32/live"
GOOD_IMAGES_DIR = "/workspaces/motion/images/esp32/good"
DISCARD_IMAGES_DIR = "/workspaces/motion/images/esp32/discard"

In [ ]:
with open(LABELS_PATH) as f:
    labels = [l for l in yaml.load(f, Loader=yaml.SafeLoader)["labels"]]

extra_labels = ['trash']
if 'discard' not in labels:
    extra_labels += ['discard']
labels += extra_labels

for d in labels:
    os.makedirs(os.path.join(INCOMING_IMAGES_DIR, d), exist_ok=True)

image_files = []
for dir, _, files in os.walk(INCOMING_IMAGES_DIR):
    for file in files:
        image_files.append((os.path.basename(dir), os.path.join(dir, file)))

In [ ]:
def do_move(img_filename, chg):
    oldpath = os.path.join(INCOMING_IMAGES_DIR, chg['old'], img_filename)
    newpath = os.path.join(INCOMING_IMAGES_DIR, chg['new'], img_filename)
    print(f"Move {img_filename} from {oldpath} to {newpath}")
    if not os.path.isdir(os.path.dirname(newpath)):
        os.makedirs(os.path.dirname(newpath), exist_ok=True)
    shutil.move(oldpath, newpath)

def do_commit(btn):
    if discard_to_trash.value is False:
        discard_target = DISCARD_IMAGES_DIR
    else:
        discard_target = os.path.join(INCOMING_IMAGES_DIR, 'trash')

    print(f"Discard target is: {discard_target}")
    for f in os.listdir(os.path.join(INCOMING_IMAGES_DIR, 'discard')):
        shutil.move(os.path.join(INCOMING_IMAGES_DIR, 'discard', f), discard_target)

    shutil.rmtree(os.path.join(INCOMING_IMAGES_DIR, 'trash'))

    for l in labels:
        if l in extra_labels:
            continue
        for f in os.listdir(os.path.join(INCOMING_IMAGES_DIR, l)):
            shutil.move(os.path.join(INCOMING_IMAGES_DIR, l, f), os.path.join(GOOD_IMAGES_DIR, l))


im_selectors = []
for i,imgfile in enumerate(image_files):
    with open(imgfile[1], "rb") as f:
        img = f.read()
    img_filename = os.path.basename(imgfile[1])
    im_widget = widgets.HBox([
        widgets.Label(value=str(i)),
        widgets.Image(value=img, format='png', width=80, height=58)
    ])
    dd = widgets.Dropdown(
        options=labels,
        value=imgfile[0],
        disabled=False,
        layout=widgets.Layout(width='95%')
    )
    dd.observe(partial(do_move, img_filename), names=['value'])
    im_selectors.append(widgets.VBox([im_widget, dd]))
pic_grid = widgets.GridBox(im_selectors, layout=widgets.Layout(grid_template_columns="repeat(10, 150px)"))

commit_btn = widgets.Button(
    description="Commit",
    icon="check"
)
commit_btn.on_click(do_commit)
discard_to_trash = widgets.Checkbox(
    value=True,
    description='Send Discard to Trash',
    disabled=False
)
commit_section = widgets.HBox([commit_btn, discard_to_trash])
widgets.VBox([pic_grid, commit_section])